In [197]:
# importing packages
import numpy as np
import pandas as pd
import sqlite3
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns


In [198]:
#create dataframe
df = pd.read_csv('honda_car_selling.csv')

In [199]:
df.head()

,Year,kms Driven,Fuel Type,Suspension,Price,Car Model
0,2019,19006 kms,Petrol,Automatic,9.29 Lakh,Honda City V CVT
1,2021,11936 kms,Petrol,Automatic,13.95 Lakh,Honda City ZX CVT
2,2018,29635 kms,Petrol,Automatic,9.95 Lakh,Honda City i-VTEC CVT ZX
3,2020,16163 kms,Petrol,Automatic,13.26 Lakh,Honda City ZX CVT
4,2015,105114 kms,Petrol,Manual,5.68 Lakh,Honda City i VTEC V


In [200]:
####data processing ##############################

In [201]:
# Splitting off units of measurement 
df[['Miles', 'kms', 'space']] = df["kms Driven"].str.split(' ', expand = True)
df[['space', 'Price (USD)', '$']] = df["Price"].str.split(' ', expand = True)

In [202]:
df.head()

,Year,kms Driven,Fuel Type,Suspension,Price,Car Model,Miles,kms,space,Price (USD),$
0,2019,19006 kms,Petrol,Automatic,9.29 Lakh,Honda City V CVT,19006,kms,,9.29,Lakh
1,2021,11936 kms,Petrol,Automatic,13.95 Lakh,Honda City ZX CVT,11936,kms,,13.95,Lakh
2,2018,29635 kms,Petrol,Automatic,9.95 Lakh,Honda City i-VTEC CVT ZX,29635,kms,,9.95,Lakh
3,2020,16163 kms,Petrol,Automatic,13.26 Lakh,Honda City ZX CVT,16163,kms,,13.26,Lakh
4,2015,105114 kms,Petrol,Manual,5.68 Lakh,Honda City i VTEC V,105114,kms,,5.68,Lakh


In [203]:
# Dropping units of measurement 
df.drop(['kms', 'kms Driven','space'], axis = 1, inplace = True)
df.drop(['Price', '$'], axis = 1, inplace = True)

In [204]:
df.head()

,Year,Fuel Type,Suspension,Car Model,Miles,Price (USD)
0,2019,Petrol,Automatic,Honda City V CVT,19006,9.29
1,2021,Petrol,Automatic,Honda City ZX CVT,11936,13.95
2,2018,Petrol,Automatic,Honda City i-VTEC CVT ZX,29635,9.95
3,2020,Petrol,Automatic,Honda City ZX CVT,16163,13.26
4,2015,Petrol,Manual,Honda City i VTEC V,105114,5.68


In [205]:
# changing lakh int to USD float 
df['Price (USD)'] = df['Price (USD)'].astype(float)
df = df[df['Price (USD)'] <= 500000]
df['Price (USD)'] = round(df['Price (USD)'] * 1216.52)
df['Price (USD)'] = df['Price (USD)'].astype(int)

In [206]:
df.shape

(992, 6)

In [207]:
df.head()

,Year,Fuel Type,Suspension,Car Model,Miles,Price (USD)
0,2019,Petrol,Automatic,Honda City V CVT,19006,11301
1,2021,Petrol,Automatic,Honda City ZX CVT,11936,16970
2,2018,Petrol,Automatic,Honda City i-VTEC CVT ZX,29635,12104
3,2020,Petrol,Automatic,Honda City ZX CVT,16163,16131
4,2015,Petrol,Manual,Honda City i VTEC V,105114,6910


In [208]:
#changing kms into miles 
df['Miles'] = df['Miles'].astype(int)
df['Miles'] = round(df['Miles'] / 1.609)
df['Miles'] = df['Miles'].astype(int)

In [209]:
df.head()

,Year,Fuel Type,Suspension,Car Model,Miles,Price (USD)
0,2019,Petrol,Automatic,Honda City V CVT,11812,11301
1,2021,Petrol,Automatic,Honda City ZX CVT,7418,16970
2,2018,Petrol,Automatic,Honda City i-VTEC CVT ZX,18418,12104
3,2020,Petrol,Automatic,Honda City ZX CVT,10045,16131
4,2015,Petrol,Manual,Honda City i VTEC V,65329,6910


In [ ]:
#convert fuel type column to boolean 
# 1 = true = petrol 
# 0 = false = diesel 
